In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bangladataset/bengali_val.jsonl
/kaggle/input/bangladataset/bengali_test.jsonl
/kaggle/input/bangladataset/bengali_train.jsonl


In [2]:
# Import necessary libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import tqdm

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("csebuetnlp/banglat5")
model = T5ForConditionalGeneration.from_pretrained("csebuetnlp/banglat5")

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import json
import os
os.environ["WANDB_DISABLED"] = "true"

# Initialize tokenizer with the model name
model_name = "csebuetnlp/banglat5"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Function to preprocess the data
def preprocess_function(examples):
    inputs = [f"summarize: {doc}" for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Define the number of entries to train with
num_entries = 2000  # For example, train with the first 2000 entries

# Load and preprocess datasets
train_dataset_path = '/kaggle/input/bangladataset/bengali_train.jsonl'
train_dataset = load_dataset('json', data_files=train_dataset_path, split=f'train[:{num_entries}]')
train_dataset = train_dataset.map(preprocess_function, batched=True)

val_dataset_path = '/kaggle/input/bangladataset/bengali_val.jsonl'
val_dataset = load_dataset('json', data_files=val_dataset_path, split='train')
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Reinitialize model
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir=f"/kaggle/working/fine_tuned_model_{num_entries}",
    num_train_epochs=5,  # Train for 5 epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f"/kaggle/working/logs_{num_entries}",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained(f"/kaggle/working/fine_tuned_model_2000")
tokenizer.save_pretrained(f"/kaggle/working/fine_tuned_model_2000")


2024-02-07 06:15:10.193287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 06:15:10.193405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 06:15:10.344014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fe07ae185a6da1d7/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

  0%|          | 0/2 [00:00<?, ?ba/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,26.432000,23.881374
2,3.014200,1.946661
3,2.496400,1.637105
4,4.204700,1.449665
5,3.754000,1.309022


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


('/kaggle/working/fine_tuned_model_2000/tokenizer_config.json',
 '/kaggle/working/fine_tuned_model_2000/special_tokens_map.json',
 '/kaggle/working/fine_tuned_model_2000/spiece.model',
 '/kaggle/working/fine_tuned_model_2000/added_tokens.json')

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from tqdm.auto import tqdm
import os

# Path for the specific fine-tuned model you want to use
model_path = "/kaggle/working/fine_tuned_model_2000"  # Example for using the model fine-tuned with 2000 entries

# Path for the test dataset
test_file_path = '/kaggle/input/bangladataset/bengali_test.jsonl'

# Function to summarize text
def summarize_text(text, model, tokenizer):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

summaries = []  # Store summaries here

# Read the test dataset and generate summaries
with open(test_file_path, 'r') as file:
    for line in tqdm(file.readlines(), desc=f"Generating summaries"):
        data = json.loads(line)
        text = data['text']  # Assuming each line contains a "text" field
        summary = summarize_text(text, model, tokenizer)
        summaries.append({"text": text, "summary": summary})

# Save the summaries to a new JSON file
output_file_path = f'/kaggle/working/summaries_after_model.json'  # Custom name for output file
with open(output_file_path, 'w') as output_file:
    json.dump(summaries, output_file, ensure_ascii=False, indent=4)

print(f"Summaries saved to {output_file_path}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating summaries:   0%|          | 0/1012 [00:00<?, ?it/s]

Summaries saved to /kaggle/working/summaries_after_model.json


In [5]:
import json

# Path to the generated summary JSON file
summary_file_path = "/kaggle/working/summaries_after_model.json"  # Adjust the filename as needed

# Function to load summaries
def load_summaries(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Load summaries from the specified file
summaries = load_summaries(summary_file_path)

# Display the first 5 samples from the set of summaries
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"Text: {summaries[i]['text']}")
    print(f"Generated Summary: {summaries[i]['summary']}")
    print("-" * 100)  # Separator for readability


Sample 1:
Text: আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক্ষোভ (ফাইল ফটো) অভিযোগ করেন জাতিসংঘের জাতিগত বৈষম্য বিষয়ক কমিটির সদস্য গে ম্যাকডুগাল। তিনি বলেন, এমন খবর তিনি পেয়েছেন যে চীনা কর্তৃপক্ষ স্বায়ত্তশাসিত উইগুর প্রদেশকে কার্যত "বিশাল একটি বন্দীশিবিরে" রূপান্তরিত করেছে। চীন সাথে সাথে এর কোনো জবাব দেয়নি। ৫০-সদস্যের চীনা প্রতিনিধিদলের পক্ষ থেকে বলা হয়, সোমবার এই অভিযোগের জবাব দেওয়া হবে। তবে এর আগে বিভিন্ন সময় চীন বলেছে, এ ধরণের বন্দী শিবিরের কোনো অস্তিত্ব নেই। বাজারে উইগুর পুরুষদের জটলা (ফাইল ফটো, ২০১৩)। চীনা কোনো বন্দীশিবিরের অস্তিত্ব অস্বীকার করে। উইগুর মুসলিম কারা? চীনের জিনজিয়াং প্রদেশের জনসংখ্যার ৪৫ শতাংশ উইগুর মুসলিম। এই প্রদেশটি তিব্বতের মত স্বশাসিত একটি অঞ্চল। বিদেশী মিডিয়ার ওপর এখানে যাবার ব্যাপারে কঠোর বিধিনিষেধ রয়েছে। কিন্তু গত বেশ ক'মাস ধরে বিভিন্ন সূত্রে খবরাখবর বেরুচ্ছে যে শিনজিয়াং-এ উইগুর এবং অন্যান্য মুসলিমরা ব্যাপকহারে আটকের শিকার হচ্ছে। ঠিক কি জন্য চীনকে অভিযুক্ত করা হচ্ছে? হিউম্যান রাইটস ওয়াচ এবং অ্যামনেস্টি ইন্টারন্যাশনাল সহ অন্যান্য আন্তর্জাতিক মানব

In [6]:
pip install rouge-score


  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a0d6d4c7ac58d77e3efa624a52d2d33545284457f838a82ebca95ab325d8544
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [7]:
from rouge_score import rouge_scorer
import json

# Load the generated summaries
generated_summaries_path = "/kaggle/working/summaries_after_model.json"  # Adjust as needed
with open(generated_summaries_path, 'r') as f:
    generated_summaries = json.load(f)

# Load the reference summaries from the test dataset
test_file_path = '/kaggle/input/bangladataset/bengali_test.jsonl'
reference_summaries = []
with open(test_file_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        reference_summaries.append(data['summary'])

# Make sure the number of generated summaries matches the number of reference summaries
assert len(generated_summaries) == len(reference_summaries), "Mismatch in number of summaries"

# Initialize the rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
rouge_scores = []
for generated, reference in zip(generated_summaries, reference_summaries):
    scores = scorer.score(reference, generated['summary'])
    rouge_scores.append(scores)

# Calculate average ROUGE scores
average_scores = {key: 0.0 for key in rouge_scores[0]}
for score in rouge_scores:
    for key in score:
        average_scores[key] += score[key].fmeasure

for key in average_scores:
    average_scores[key] /= len(rouge_scores)

print("Average ROUGE scores:")
for key, value in average_scores.items():
    print(f"{key}: {value:.4f}")


Average ROUGE scores:
rouge1: 0.0000
rouge2: 0.0000
rougeL: 0.0000
